<a href="https://colab.research.google.com/github/dercodeKoenig/hft_dqn/blob/main/dqn_tpu_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/dercodeKoenig/hft_dqn

fatal: destination path 'hft_dqn' already exists and is not an empty directory.


In [2]:
%cd hft_dqn

/content/hft_dqn


In [4]:
!kaggle datasets download bpwqsdd/us-futures-1-minute-candlesticks
!unzip us-futures-1-minute-candlesticks.zip

Dataset URL: https://www.kaggle.com/datasets/bpwqsdd/us-futures-1-minute-candlesticks
License(s): unknown
us-futures-1-minute-candlesticks.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  us-futures-1-minute-candlesticks.zip
replace ES_2? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ES_2                    
  inflating: NQ_2                    
  inflating: YM_2                    


In [3]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import multiprocessing as mp
from datetime import datetime, timedelta
import time
from collections import deque
import numpy as np
from tqdm import tqdm
import copy
import tensorflow as tf
import random
from tqdm import tqdm
from save_and_load import *
from Candle import Candle
from MultiTimeframeCandleManager import MultiTimeframeCandleManager

start_time = time.time()

In [4]:
try:
        cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu="local")
        tf.config.experimental_connect_to_cluster(cluster_resolver)
        tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
        strategy = tf.distribute.TPUStrategy(cluster_resolver)
        print("use tpu strategy")
except:
        print("use gpu strategy")
        strategy = tf.distribute.MirroredStrategy()

use tpu strategy


In [5]:
gamma = 0.99
batch_size = 1024

path = "/content/drive/MyDrive/dqn_t1/"

ep_len = 1000

m1 = np.eye(2, dtype="float32")
num_model_inputs = 6

In [6]:
with strategy.scope():

  chart_m15 = tf.keras.layers.Input(shape = (60,4))
  chart_m5 = tf.keras.layers.Input(shape = (60,4))
  chart_m1 = tf.keras.layers.Input(shape = (60,4))

  pdas = tf.keras.layers.Input(shape = (19,))

  current_position = tf.keras.layers.Input(shape = (1,))
  #scaled_open_profit = tf.keras.layers.Input(shape = (1,))

  minutes = tf.keras.layers.Input(shape = (1,))
  minutes_embed = tf.keras.layers.Embedding(input_dim=60*24, output_dim=8)(minutes)
  minutes_embed_flat = tf.keras.layers.Flatten()(minutes_embed)

  f15 = tf.keras.layers.Flatten()(chart_m15)
  f5 = tf.keras.layers.Flatten()(chart_m5)
  f1 = tf.keras.layers.Flatten()(chart_m1)

  #c = tf.keras.layers.Concatenate()([f15, f5, f1, pdas, minutes_embed_flat, current_position, scaled_open_profit])
  c = tf.keras.layers.Concatenate()([f15, f5, f1, pdas, minutes_embed_flat, current_position])

  lrelu = tf.keras.layers.LeakyReLU(0.05)

  d = tf.keras.layers.Dense(1024*2)(c)
  d = lrelu(d)
  d = tf.keras.layers.Dense(1024*2)(d)
  d = lrelu(d)

  value = tf.keras.layers.Dense(1, activation="linear")(d)
  advantage = tf.keras.layers.Dense(2, activation="linear")(d)

  q_values = tf.keras.layers.Lambda(
    lambda inputs: inputs[0] + (inputs[1] - tf.reduce_mean(inputs[1], axis=1, keepdims=True))
  )([value, advantage])



  model = tf.keras.Model(inputs = [chart_m15, chart_m5, chart_m1, pdas, minutes, current_position], outputs = q_values)
  target_model = tf.keras.Model(inputs = [chart_m15, chart_m5, chart_m1, pdas, minutes, current_position], outputs = q_values)

  optimizer = tf.keras.optimizers.Adam(learning_rate = 0.000001)


model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5             │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer (InputLayer)  │ (None, 60, 4)          │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 60, 4)          │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_2             │ (None, 60, 4)          │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 1, 8)           │         11,520 │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_1 (Flatten)       │ (None, 240)            │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_2 (Flatten)       │ (None, 240)            │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_3 (Flatten)       │ (None, 240)            │              0 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_3             │ (None, 19)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 8)              │              0 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_4             │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 748)            │              0 │ flatten_1[0][0],       │
│                           │                        │                │ flatten_2[0][0],       │
│                           │                        │                │ flatten_3[0][0],       │
│                           │                        │                │ input_layer_3[0][0],   │
│                           │                        │                │ flatten[0][0],         │
│                           │                        │                │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 2048)           │      1,533,952 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu (LeakyReLU)   │ (None, 2048)           │              0 │ dense[0][0],           │
│                           │                        │                │ dense_1[0][0]          │
├──────────────────────

 Total params: 5,747,971 (21.93 MB)

 Trainable params: 5,747,971 (21.93 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:

def relative (value, center, r):
        return (value - center) / r

def ret_to_scaled_inputs(ret):

    midnight_open, midnight_opening_range_high,midnight_opening_range_low, pdas, current_close, current_time, charts = ret

    center = (midnight_opening_range_high + midnight_opening_range_low) / 2
    r = max(1,(midnight_opening_range_high - midnight_opening_range_low) / 2)

    pda_rel = []
    pda_rel.append(relative(midnight_open, center, r))
    for pda in pdas:
        pda_rel.append(relative(pda, center, r))
    pda_np = np.array(pda_rel)

    current_minutes = current_time.hour * 60 + current_time.minute

    charts_array = []
    for candlesticks in charts:
        charts_array.append([])
        for candle in candlesticks:
            o = relative(candle.o, center, r)
            h = relative(candle.h, center, r)
            l = relative(candle.l, center, r)
            c = relative(candle.c, center, r)
            charts_array[-1].append([o,h,l,c])

    m15_np = np.array(charts_array[0])
    m5_np = np.array(charts_array[1])
    m1_np = np.array(charts_array[2])

    return [m15_np, m5_np, m1_np, pda_np, current_minutes]

In [8]:
def generate_data(candles, cmm):

    m = MultiTimeframeCandleManager()

    last_close = 0
    last_state = None
    last_action = 0

    sarts = []

    for index in range(len(candles)):

        ret = m.push_m1_candle(candles[index])
        midnight_open, midnight_opening_range_high,midnight_opening_range_low, pdas, current_close, current_time, charts = ret
        center = (midnight_opening_range_high + midnight_opening_range_low) / 2
        r = max(1,(midnight_opening_range_high - midnight_opening_range_low) / 2)


        if(len(m.m15_candles) == 60):

            state = ret_to_scaled_inputs(ret)
            m15_np, m5_np, m1_np, pda_np, current_minutes = state

            if(last_state != None):

                terminal = 0
                if(index+1 == len(candles)):
                    terminal = 1

                current_close = state[2][-1][3]
                last_close = last_state[2][-1][3]

                ### last position -1
                state_short = last_state + [-1]
                ## action was -1
                new_state = state + [-1]
                reward = ((current_close - last_close) * -1) / r
                sarts.append([state_short, 0, reward, terminal, new_state])
                ## action was 1
                new_state = state + [1]
                reward = ((current_close - last_close) * 1 - cmm) / r
                sarts.append([state_short, 1, reward, terminal, new_state])


                ### last position 1
                state_long = last_state + [1]
                ## action was -1
                new_state = state + [-1]
                reward = ((current_close - last_close) * -1 - cmm) / r
                sarts.append([state_long, 0, reward, terminal, new_state])
                ## action was 1
                new_state = state + [1]
                reward = ((current_close - last_close) * 1) / r
                sarts.append([state_long, 1, reward, terminal, new_state])


            last_state = state

    return sarts

In [9]:
def process_candles(file_path, multiplier):
    candles = obj_load(file_path)
    return generate_data(candles, multiplier)

tasks = [
    ("NQ_2", 0.5),
    ("ES_2", 0.5),
    ("YM_2", 1.5)
]

# Create a pool of workers
with mp.Pool(processes=len(tasks)) as pool:
    # Map tasks to the pool
    results = pool.starmap(process_candles, tasks)

# Combine the results into sarts_memory
sarts_memory = []
for result in tqdm(results):
        sarts_memory.extend(result)

del results

loadingloadingloading   NQ_2YM_2ES_2




100%|██████████| 3/3 [00:01<00:00,  2.48it/s]


In [20]:
sarts_memory = deque(sarts_memory)

batches = []
num_batches = int(len(sarts_memory) / batch_size)

for _ in tqdm(range(num_batches)):

    sarts_sample = [sarts_memory.popleft() for _ in range(batch_size)]  # Fast removal

    states = [x[0] for x in sarts_sample]
    actions = [x[1] for x in sarts_sample]
    rewards = np.array([x[2] for x in sarts_sample], dtype="float32")
    terminals = np.array([x[3] for x in sarts_sample], dtype="float32")
    next_states = [x[4] for x in sarts_sample]

    next_states_array = []
    for i in range(num_model_inputs):
        next_states_array.append(np.array([x[i] for x in next_states], dtype = "float32"))


    states_array = []
    for i in range(num_model_inputs):
        states_array.append(np.array([x[i] for x in states], dtype = "float32"))


    masks = np.array(m1[actions], dtype="float32")

    batches.append([states_array, masks, rewards, terminals, next_states_array])

del sarts_memory

100%|██████████| 23118/23118 [03:11<00:00, 120.57it/s]


In [48]:
@tf.function()
def get_target_q(next_states, rewards, terminals):
            estimated_q_values_next = target_model(next_states)
            q_batch = tf.math.reduce_max(estimated_q_values_next, axis=1)
            target_q_values = q_batch * gamma * (1-terminals) + rewards
            return target_q_values

@tf.function()
def tstep(data):
    states, masks, rewards, terminals, next_states = data

    target_q_values = get_target_q(next_states, rewards, terminals)

    with tf.GradientTape() as t:
        model_return = model(states, training=True)
        mask_return = model_return * masks
        estimated_q_values = tf.math.reduce_sum(mask_return, axis=1)
        #print(estimated_q_values, mask_return, model_return, masks)
        loss_e = tf.math.square(target_q_values - estimated_q_values)
        loss = tf.reduce_mean(loss_e)


    gradient = t.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradient, model.trainable_variables))

    return loss, tf.reduce_mean(estimated_q_values)


batch_index = 0
def tpu_data_get_func(_n):
    global batch_index
    batch_index+=1
    if(batch_index >= len(batches)):
        batch_index = 0
    return batches[batch_index]


def run():

    distributed_data = (strategy.experimental_distribute_values_from_function(tpu_data_get_func))
    loss, q = strategy.reduce(tf.distribute.ReduceOp.MEAN, strategy.run(tstep, args = (distributed_data,)), axis = None)

    return loss, q

In [49]:
model_save_name = "model_small.weights.h5"


loss_mean = []
q_mean = []

try:
    model.load_weights(path+model_save_name)
    target_model.load_weights(path+model_save_name)
    print("loaded weights")
    loss_mean = obj_load(path+"loss")
    q_mean = obj_load(path+"q")
except Exception as e:
    print(e)
    print("unable to load data")

def save():
            model.save_weights(path+model_save_name)
            obj_save(loss_mean, path+"loss")
            obj_save(q_mean, path+"q")
            print("saved progress")
            upload_model_to_server()


from ftplib import FTP

def upload_model_to_server():
    # FTP server details
    ftp_server = 'benundmarvpromotions.lima-ftp.de'
    ftp_user = 'benundmarvpromotions'
    ftp_password = 'gWEhrtjanrgy'

    file_to_upload = path+model_save_name
    remote_path = '/test/' + model_save_name

    # Connect to the FTP server
    ftp = FTP(ftp_server)
    ftp.login(user=ftp_user, passwd=ftp_password)

    ftp.cwd("/")

    # Open the file in binary mode and upload it
    with open(file_to_upload, 'rb') as file:
        ftp.storbinary(f'STOR {remote_path}', file, blocksize = 1024*1024)

    # Close the connection
    ftp.quit()

    print("File uploaded successfully!")

loaded weights
loading /content/drive/MyDrive/dqn_t1/loss
loading /content/drive/MyDrive/dqn_t1/q


In [ ]:

safe_after_eps = 30
eps_counter=0

while True:
    eps_counter+=1
    try:
        loss = []
        q = []
        rewards_tmp = []
        actions = []
        progbar = tf.keras.utils.Progbar(ep_len)
        for i in range(ep_len):

            c_loss, c_q = run()
            loss.append(c_loss)
            q.append(c_q)

            #progbar.update(i+1, values = [("loss", c_loss), ("qv", c_q)])

        loss_mean.append(np.mean(loss))
        q_mean.append(np.mean(q))

        progbar.update(ep_len, values = [("loss", np.mean(loss)), ("qv", np.mean(q))])

        target_model.set_weights(model.get_weights())

        if(eps_counter >= safe_after_eps):
            eps_counter = 0
            save()

        if time.time() - start_time > 60*60*8.5:
            break


    except    KeyboardInterrupt:
        print("")
        print("exit")
        save()
        break


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 55s 55ms/step - loss: 12.5641 - qv: 62.4977
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 49s 49ms/step - loss: 63.9925 - qv: 48.1112
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 50s 50ms/step - loss: 95.4309 - qv: 57.0879
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 49s 49ms/step - loss: 883.5432 - qv: 61.0321
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 55s 55ms/step - loss: 30.3170 - qv: 53.7789
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 60s 60ms/step - loss: 54.0432 - qv: 64.0708
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 49s 49ms/step - loss: 23.1816 - qv: 59.7661
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 51s 51ms/step - loss: 84129.6719 - qv: 214.4107
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 51s 51ms/step - loss: 68.3849 - qv: 66.0445
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 49s 49ms/step - loss: 49.1753 - qv: 64.4407
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 49s 49ms/step - loss: 36.2136 - qv: 62.2974
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 49s 49ms/step - loss: 45.9519 - qv: 70.0846
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 49s 49ms/step - loss: 23.7037 - qv: 63.3620
1000/1000 ━━━━━━━━━━